In [1]:
import subprocess
import os
import time
import csv

TIMEOUT = 120
REPETITIONS = 5
ENUMERATION_LIMIT = 1000

In [2]:
# Runs trapmvn with either unitary or general update scheme,
# reporting the runtime, number of fails, and the number of results.
def bench_trapmvn(model, semantics = "general"):
    cmd_trapmvn = ["python3", "-m", "trapmvn", "-c", "min", "-s", semantics, "-m", str(ENUMERATION_LIMIT), model]
    total_time = 0
    fails = 0
    result_count = 0    
    for _i in range(REPETITIONS):
        try:
            start = time.perf_counter()
            result = subprocess.run(cmd_trapmvn, capture_output=True, timeout=TIMEOUT)
            total_time += time.perf_counter() - start
            result_count = len(result.stdout.splitlines()) - 1
            if result.returncode != 0:
                fails += 1
        except subprocess.TimeoutExpired:
            total_time += TIMEOUT
            fails += 1
    return (float(total_time) / float(REPETITIONS), fails, result_count)

In [3]:
# Compute minimal trap spaces using trappist and measure the outcomes.
def bench_trappist(model):
    model_bnet = model.replace(".sbml", ".bnet")
    cmd_trappist = ["python3", "-m", "trappist", "-s", "asp", "-c", "min", "-m", str(ENUMERATION_LIMIT), model_bnet]
    total_time = 0
    fails = 0
    result_count = 0    
    for _i in range(REPETITIONS):
        try:
            start = time.perf_counter()
            # First, create a bnet model file.
            result = subprocess.call(['java', '-jar', 'bioLQM.jar', model, model_bnet], stdout=subprocess.DEVNULL)
            assert result == 0
            # Then run trappist on this bnet file.
            result = subprocess.run(cmd_trappist, capture_output=True, timeout=TIMEOUT)
            total_time += time.perf_counter() - start            
            result_count = len(result.stdout.splitlines()) - 1            
            if result.returncode != 0:
                fails += 1
        except subprocess.TimeoutExpired:
            total_time += TIMEOUT
            fails += 1
    return (float(total_time) / float(REPETITIONS), fails, result_count) 

In [4]:
# Compute minimal trap space using mpbn and measure the outcomes.
def bench_mpbn(model):
    model_bnet = model.replace(".sbml", ".bnet")
    total_time = 0
    fails = 0
    result_count = 0
    for _i in range(REPETITIONS):
        try:
            start = time.perf_counter()
            # First, convert .sbml to .bnet
            result = subprocess.call(['java', '-jar', 'bioLQM.jar', model, model_bnet], stdout=subprocess.DEVNULL)
            assert result == 0
            # Then use mpbn
            output = subprocess.run(['python3', 'mpbn_helper.py', model_bnet, "min", str(ENUMERATION_LIMIT)], capture_output=True, timeout=TIMEOUT)
            total_time += time.perf_counter() - start            
            result_count = len(output.stdout.splitlines())  
            if output.returncode != 0:
                fails += 1
        except subprocess.TimeoutExpired:
            total_time += TIMEOUT
            fails += 1
    return (float(total_time) / float(REPETITIONS), fails, result_count) 

In [ ]:
# For maximal trap spaces, we can only test trapmvn and nothing else.
# But we can still test general/unitary semantics.
with open('min-trap-benchmark.tsv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter='\t')
    
    header = ["Model"]
    header += ["trapmvn (general)", "trapmvn (unitary)"]
    header += ["trappist", "mpbn"]
    writer.writerow(header)
    print(header)
    
    for model in sorted(os.listdir("models")):
        if not model.endswith(".sbml"):
            continue
        
        row = [model]
        model = f"models/{model}"
        # Fixed points are shared across semantics.
        trapmvn_general = bench_trapmvn(model, semantics="general")
        trapmvn_unitary = bench_trapmvn(model, semantics="unitary")        
        row += [str(trapmvn_general), str(trapmvn_unitary)]
        trappist = bench_trappist(model)
        mpbn = bench_mpbn(model)
        row += [str(trappist), str(mpbn)]
        print(row)
        
        writer.writerow(row)
        csvfile.flush()